# OpenAI Introduction

Use this for [reference](https://platform.openai.com/docs/api-reference/authentication)

In [ ]:
%pipenv install openai python-dotenv -q

> Create an OpenAI account and get an API_KEY

### OpenAI:

1. Chat completion API ✅
2. Implement structured output ✅
3. Implement tool calling ✅
4. Implement streaming ✅
5. Implement assistant ✅
6. Response API ✅
7. Chaining of response ✅

In [22]:
from openai import AzureOpenAI
from dotenv import load_dotenv
from os import getenv

load_dotenv()

True

In [23]:
client = AzureOpenAI(
    api_key=getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=getenv("AZURE_OPENAI_API_VERSION")
)

### List Models

In [16]:
models = client.models.list()
model_names = [model.id for model in models if "gpt" in model.id]

In [10]:
model_names

['gpt-35-turbo-0301',
 'gpt-35-turbo-0613',
 'gpt-35-turbo-1106',
 'gpt-35-turbo-0125',
 'gpt-35-turbo-instruct-0914',
 'gpt-35-turbo-16k-0613',
 'gpt-4-0125-Preview',
 'gpt-4-1106-Preview',
 'gpt-4-0314',
 'gpt-4-0613',
 'gpt-4-32k-0314',
 'gpt-4-32k-0613',
 'gpt-4-vision-preview',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-turbo-jp',
 'gpt-4o-2024-05-13',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini-2024-07-18',
 'gpt-4o-2024-11-20',
 'gpt-4o-audio-mai',
 'gpt-4o-realtime-preview',
 'gpt-4o-mini-realtime-preview-2024-12-17',
 'gpt-4o-realtime-preview-2024-12-17',
 'gpt-4o-canvas-2024-09-25',
 'gpt-4o-audio-preview-2024-10-01',
 'gpt-4o-audio-preview-2024-12-17',
 'gpt-4o-mini-audio-preview-2024-12-17',
 'gpt-4o-transcribe-2025-03-20',
 'gpt-4o-mini-transcribe-2025-03-20',
 'gpt-4o-mini-tts-2025-03-20',
 'gpt-4.1-2025-04-14',
 'gpt-4.1-mini-2025-04-14',
 'gpt-4.1-nano-2025-04-14',
 'gpt-35-turbo',
 'gpt-35-turbo-instruct',
 'gpt-35-turbo-16k',
 'gpt-4',
 'gpt-4-32k',
 'gpt-4o',
 'gpt-4o-mini',
 'gpt-4

| 1. Chat Completion |
|--|


The Chat Completions API endpoint will generate a model response from a list of messages comprising a conversation.

In [11]:
MODEL_NAME = "gpt-4o-mini"

In [12]:
completion = client.chat.completions.create(
  model=MODEL_NAME,
  store=False,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

In [13]:
print(completion.choices[0].message.content)

Lines of code, they hum,  
Dreaming in electric thoughts—  
Future blooms in light.


| 2. Structured Output |
|--|

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema,

In [ ]:
!pipenv install pydantic

In [ ]:
from pydantic import BaseModel

In [ ]:
class UserRegistration(BaseModel):
    name: str
    age: int
    tier: int

In [ ]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the user information."},
        {"role": "user", "content": "We have a new user named Tim aged 25 with a tier 3 account."},
    ],
    response_format=UserRegistration,
)

In [ ]:
event = completion.choices[0].message.parsed
event

| 3. Tool/Function Calling |
|--|

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services. 



In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

| Function -> Tool JSON Convertor |
|--|

Automate tool JSON creation

In [ ]:
import inspect
from typing import Callable, get_type_hints

In [ ]:
def generate_json_schema(func: Callable) -> str:
    """
    Generate a JSON schema for a given function based on its docstring and type hints.

    Args:
        func (Callable): The Python function to generate a schema for.

    Returns:
        str: A JSON string representing the function's schema.

    Raises:
        ValueError: If the function lacks a docstring.
    """
    # Get function metadata
    if not func.__doc__:
        raise ValueError("Function must have a docstring")

    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    # Parse docstring
    docstring = inspect.getdoc(func) or ""
    doc_lines = docstring.split('\n')
    description = doc_lines[0].strip()  # First line is the main description
    param_descriptions = {}
    in_args_section = False

    # Extract parameter descriptions
    for line in doc_lines:
        line = line.strip()
        if line.lower().startswith('args:'):
            in_args_section = True
            continue
        if in_args_section and line and ':' in line:
            # Handle lines like "param_name (type): description"
            parts = line.split(':', 1)
            param_part = parts[0].strip()
            desc = parts[1].strip() if len(parts) > 1 else ""
            # Extract param_name from "param_name (type)"
            if '(' in param_part and ')' in param_part:
                param_name = param_part[:param_part.index('(')].strip()
            else:
                param_name = param_part
            if param_name:
                param_descriptions[param_name] = desc

    # Map Python types to JSON schema types
    type_mapping = {
        'str': 'string',
        'int': 'integer',
        'float': 'number',
        'bool': 'boolean',
        'dict': 'object',
        'list': 'array'
    }

    # Build schema
    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    # Add parameters to schema
    for param_name, param in signature.parameters.items():
        # Get type from type hints, default to str if not specified
        param_type = type_hints.get(param_name, str).__name__
        schema_type = type_mapping.get(param_type, 'string')  # Default to string
        
        schema["function"]["parameters"]["properties"][param_name] = {
            "type": schema_type,
            "description": param_descriptions.get(param_name, f"{param_name} parameter")
        }
        # Mark as required if no default value
        if param.default == inspect.Parameter.empty and param.kind != inspect.Parameter.VAR_KEYWORD:
            schema["function"]["parameters"]["required"].append(param_name)

    return schema

#### Weather function

In [ ]:
def get_weather(location: str) -> dict:
    """
    Get current temperature for a given location.

    Args:
        location (str): City and country e.g. Bogotá, Colombia

    Returns:
        dict: A dictionary containing the current temperature and other weather details.
    """
    return {"location": location, "temperature": 25, "unit": "Celsius"}

In [ ]:
generate_json_schema(func=get_weather)

In [ ]:
tools[0]

#### Tool Calling

In [ ]:
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

In [ ]:
print(completion.choices[0].message.tool_calls)

| 4. Streaming |
|--|


Streaming responses lets you start printing or processing the beginning of the model's output while it continues generating the full response.

In [ ]:
stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user",
            "content": "Say 'double bubble bath' ten times fast.",
        },
    ],
    stream=True,
)

In [ ]:
for chunk in stream:
    print(chunk)
    print(chunk.choices[0].delta)
    print("****************")

| Assistants |
|--|

Build assistants that can call models and use tools to perform tasks.

In [ ]:
my_assistant = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    name="Math Tutor",
    tools=[{"type": "code_interpreter"}],
    model=MODEL_NAME
)

In [ ]:
print(my_assistant.id)

In [17]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_rNGvA6U6tKtWIRKEYZPo5W7t', created_at=1739434286, description=None, instructions='# **Context**  \nYou are **Ava**, a professional and friendly AI chat assistant representing **Amplity**. Amplity is a full-service partner delivering flexible and specialized medical and commercial services. Amplity supports clients across all stages of the drug lifecycle, scaling with ease to maximize resources and improve impact.  Amplity has five services named as following:\n- [Amplity Medical](https://amplity.com/medical)\n- [Amplity Sales](https://amplity.com/sales)\n- [Amplity Intel](https://amplity.com/intel)\n- [Amplity Comms](https://amplity.com/comms)\n- [Amplity Learn](https://amplity.com/learn)\n\nAmplity operates the following lines of business: Comms(headed by Susan Duffy), Intel (distinct from Business Intelligence)(headed by Michele Graham), Learn(headed by Michele Graham), Medical(headed by Denise Chambley) and  Sales(headed by Brian O\'Donnell). Amplity leaderships 

In [19]:
my_assistant = client.beta.assistants.retrieve(assistant_id="asst_rNGvA6U6tKtWIRKEYZPo5W7t")

In [20]:
client.beta.assistants.delete(assistant_id="asst_rNGvA6U6tKtWIRKEYZPo5W7t")

AssistantDeleted(id='asst_rNGvA6U6tKtWIRKEYZPo5W7t', deleted=True, object='assistant.deleted')

Create a Thread

In [ ]:
thread = client.beta.threads.create()
thread_id = thread.id
print(f"Thread ID: {thread_id}")

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="Explain factorial?"
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=my_assistant.id
)

# Wait for the run to complete
import time
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run_status.status == "completed":
        break
    time.sleep(1)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
for message in messages.data:
    if message.role == "assistant":
        print(f"Assistant: {message.content[0].text.value}")

| 6. Response API |
|--|

Allow the model access to external systems and data using function calling.

We get the option to outsource that to OpenAI entirely: you can add a new "store": true property and then in subsequent messages include a "previous_response_id: response_id key to continue that conversation.

In [ ]:
response = client.responses.create(
    model=MODEL_NAME,
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Response API with Tools

In [ ]:
response = client.responses.create(
    model=MODEL_NAME,
    tools=[{"type": "web_search_preview"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

| 7. Response Chaining |
|--|

In [24]:
first_prompt = "Generate a one-sentence idea for a short story about a time traveler."
response1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": first_prompt}
    ]
)

# Extract the output from the first call
story_idea = response1.choices[0].message.content
print("First response (story idea):", story_idea)

# Second API call: Use the first response to expand into a brief outline
second_prompt = f"Create a brief outline for a short story based on this idea: {story_idea}"
response2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": second_prompt}
    ]
)

# Extract and print the second response
story_outline = response2.choices[0].message.content
print("\nSecond response (story outline):\n", story_outline)

First response (story idea): When a disillusioned historian discovers a way to travel back to pivotal moments in history, she must choose between altering a tragic event for the better or preserving the timeline, knowing that her decision could erase her own existence.

Second response (story outline):
 **Title:** Echoes of Time

**Outline:**

**I. Introduction**
- **A. Protagonist:** Introduce Rachel, a disillusioned historian specializing in tragic events. She feels her work is futile as history tends to repeat itself.
- **B. Discovery:** Rachel stumbles upon an ancient artifact in a dusty library that allows time travel to pivotal moments in history.
- **C. Motivation:** Driven by personal loss due to a tragic event (e.g., a war or natural disaster), Rachel is tempted to change history for the better.

**II. First Journey: A Test Run**
- **A. Excursion:** Rachel travels to a minor historical event to test the artifact, experiencing the thrill of being a direct observer.
- **B. Conse